In [48]:
# Library Import

import pandas as pd
import numpy as np
import requests
import os
import datetime as dt
import requests
import json
import io

# Set Program Parameters

days_previous = 7

In [2]:
# I have already gathered existing Wind (Forecast & Actual), Demand , Interconnection data. Portal currently allows 
# user to gather data up to 2 months in the past.

# merge the dataframes

merged_df = pd.read_csv("system_demand.csv", parse_dates = ['DATE & TIME'])

work_dir = os.getcwd()

for file in os.listdir(work_dir):
    if file.endswith("tion.csv"):
        df = pd.read_csv(file, parse_dates = ['DATE & TIME'])
        merged_df = pd.merge(merged_df, df, how = 'left', on = ['DATE & TIME'])
        
# Drop & rename irrelevant columns

merged_df = merged_df.drop([' FORECAST DEMAND(MW)', ' REGION_x', ' NET TOTAL(MW)', ' REGION_y'], axis = 1)
cols = merged_df.columns.to_list()
merged_df = merged_df.rename(columns = {cols[0]: 'date_time_bst', cols[1]: 'demand_mw', cols[2]: 'ewic_mw', 
                                        cols[3]: 'moyle_mw',cols[4]: 'generation_mw', cols[5]: 'forecast_wind_mw',
                                        cols[6]: 'actual_wind_mw', cols[7]: 'region'})

merged_df.to_excel('combined_eirgrid_data.xlsx', index = False)

In [49]:
def get_data(start_day, fund_name):
    """
    Enables user to download data from the portal.
    Requires two arguments 'start_day' (int): specifies how far back the user wants to start collecting data
                          'fund_name' (str): specifies the fundamental to be gathered i.e. demandActual, generationActual,
                                             windActual or interconnection
    """
    today = dt.date.today()
    start_date = today - dt.timedelta(days = start_day)
    start_date = dt.datetime.strftime(start_date, "%d-%b-%Y")
    end_date = today - dt.timedelta(days = 1)
    end_date = dt.datetime.strftime(end_date, "%d-%b-%Y")
    
    eirgrid_url = "http://smartgriddashboard.eirgrid.com/DashboardService.svc/csv?area="+fund_name+"&region=ALL&datefrom="+start_date+"%2000:00&dateto="+end_date+"%2023:59"
    
    try:
        r = requests.get(eirgrid_url)
    except Exception as e:
        print("Error acquiring data: %s", str(e))
        return
    
    return r.content

In [50]:
def process_data(fund_name):
    """
    Output from get_data function is passed.
    Processes the data into a dataframe, an appropriate format for efficient storage
    Requires one argument: 'fund_name' (str): specifies the fundamental to be gathered i.e. demandActual, 
                                              generationActual, windActual or interconnection
    """
        
    data = get_data(days_previous, fund_name)
    data = pd.read_csv(io.StringIO(data.decode('utf-8')))
    data['DATE & TIME'] = data['DATE & TIME'].apply(lambda x: dt.datetime.strptime(x, '%d %B %Y %H:%M'))
    return data

In [56]:
def main():
    """
    Initiates the program (calls process_data and then get_data())
    Writes each fundamental's data to the desired .csv
    """
    
    fund_list = ['demandActual', 'generationActual', 'windActual', 'interconnection']
    
    try:
        for fund in fund_list:
        
            data = process_data(fund)
            if fund == 'demandActual':
                data.to_csv('system_demand.csv', mode='a', header=False, index=False)
            elif fund == 'generationActual':
                data.to_csv('system_generation.csv', mode='a', header=False, index=False)
            elif fund == 'windActual':
                data.to_csv('wind_generation.csv', mode='a', header=False, index=False)
            elif fund == 'interconnection':
                data.to_csv('interconnection.csv', mode='a', header=False, index=False)
            else:
                print('Error: Fundamental not available')
    except Exception as e:
        print(str(e))             